<h2 style="color: rgb(241, 90, 36)"><img src="./images/SQLIcon.png?modified=223" width=80px height=80px style="vertical-align: middle;">CRUD Altering Tables</h2>

After you have inserted data into your database tables, there of course might be a need to alter, or insert new data. In this notebook you will learn how you can update your tables using **CRUD operations**.

## Adding/removing columns

When updating a table you can add or drop columns, rows or completely change data points, as long as the data matches the correct type and the constraints are met. 

Manipulations of columns are done using the `ALTER TABLE` statement with the following syntax:

In [ ]:
ALTER TABLE {table_name} 
    ADD COLUMN {column name} {data_type} {constraint};

Adding a new `available` column to signify that a film is available for rental:

In [ ]:
ALTER TABLE rental
    ADD COLUMN available BOOLEAN NOT NULL DEFAULT True;

Notice the additional statement keyword `DEFAULT` was added with the value set to `True`. Since we set column constraint to be `NOT NULL`, we needed to set the default value as the column can't contain `NULL` values. SQL would throw an error if we didn't set this default value. 

Dropping columns can also be done using the syntax:

In [ ]:
ALTER TABLE {table_name}
    DROP COLUMN {column_name} [ RESTRICT | CASCADE ];

Again we can choose to cascade the dropping of constraints and relationships if another table depends on the column.

In [ ]:
ALTER TABLE rental
    DROP COLUMN available;

<h3 style="color: rgb(241, 90, 36)">Renaming columns and tables</h3>

You might want to rename columns or tables if they no longer make sense to the current needs of the business.

To change this name of a table use the following syntax:

In [ ]:
ALTER TABLE {table_name}
    RENAME TO {new_table_name};

Column renaming can be done with:

In [ ]:
ALTER TABLE {table_name}
    RENAME {previous_column_name} TO {new_column_name};

<h3 style="color: rgb(241, 90, 36)">Changing column types</h3>

Often column types may need to be changed, this could be due to the current data type not handling the size of the data correctly. The data type could be too small to hold the data or too large, taking up extra space. Sometimes on first insertion of data into a new table by another service or application the data types might be inferred. This can lead to data being modelled incorrectly and thus might need to be altered.

The syntax to alter the column data type is:


In [ ]:
ALTER TABLE {table_name}
    ALTER COLUMN {column_name} TYPE {data type};

We could remove the time zone from the `rental_date` column in the `rental` table by casting it to a timestamp without time zone:

In [ ]:

ALTER TABLE rental
    ALTER COLUMN rental_date TYPE TIMESTAMP without TIME ZONE;

Although casting is great for quickly converting column types, it won't work in all cases. The current data type to be cast has to be of the correct form to be cast to the new value. For example you can't cast a `timestamp` to an `integer`, but casting an `integer` to another `numerical` type like a `real` would be possible.

## Adding/Removing constraints

Constraints can also be altered after the table was created using `ALTER TABLE`. This can be used to both add and remove table constraints.

To remove constraints use the following syntax:


In [ ]:
ALTER TABLE {table_name}
    DROP CONSTRAINT {name_of_the_constraint} [ RESTRICT | CASCADE]

Table constraints can be added with:

In [ ]:
ALTER TABLE {table_name}
    ADD {table_constraint}

For example adding a constraint that the `staff_id > 0` to the `staff` table:

In [ ]:
ALTER TABLE staff
    ADD CHECK (staff_id > 0)

-- Or with a named constraint, always better to name your constraint as best practice

ALTER TABLE staff
    ADD CONSTRAINT min_staff_id CHECK (staff_id > 0)

## Adding new rows

To add new rows to a table we can use the following syntax: 

In [ ]:
INSERT INTO {table_name} 
    ({column_1}, {column_2}, ....)
VALUES 
    ({column_1 value}, {column_2 value}, ......);

Let's insert a new film category into the `category` table:

In [ ]:
INSERT INTO category
    (name, last_update)
VALUES 
    ('Thriller', NOW());

Important to note here that the order of `(name, last_update)` only matters when referencing the values to be inserted. In the `VALUES` clause we have input `('Thriller', NOW())` to insert the category `'Thriller'` at the current datetime `NOW()`. Since `name` comes first in the `INSERT INTO` clause `Thriller` will be inserting into the `name` column first and the timestamp `NOW()` inserted into the `last_update` column as it comes second. 

This order doesn't need to reflect the ordering of the table. It just needs to be ordered in the correct way in the `INSERT INTO` and `VALUES` clauses, better seen with an example:

In [ ]:
INSERT INTO category
    (last_update, name)
VALUES 
    (NOW(), 'Thriller');

This would insert the same data, the positions of the arguments only matter in the statement itself. We also didn't include the column `id` since SQL will auto generate the `id` on table insertion.

<h4 style="color: rgb(241, 90, 36)">Inserting rows with SELECT</h4>

SQL also gives you the ability to insert rows into tables using the `SELECT` statement. The syntax is a little different from inserting rows using the `VALUES` clause. Rows can be inserted with the `SELECT` statement using the syntax:


In [ ]:
INSERT INTO {table_name} 
    ({column_1}, {column_2}, ....)
(SELECT query);

Let's create a small table in the database which will contain films starting with the letter B and a few details about the films. We will use the `film` table to get the data to insert using a `SELECT` statement:

In [ ]:
CREATE TABLE b_films (
    title VARCHAR(300) UNIQUE NOT NULL,
    description TEXT NOT NULL,
    release_year NOT NULL,
    rating CHAR(5) NOT NULL
);

Now we can insert the data using `SELECT`. When selecting the data we need to select the same number of columns we want to insert and of the correct data type to insert or SQL will cause an error. 

In [ ]:
INSERT INTO B_films(title, description, release_year, rating)
(SELECT title, 
        description, 
        release_year,
        rating
    FROM
        film
    WHERE 
        title LIKE 'B%
);

This fills the `b_films` table with all films starting with the letter B. You can add as much complexity to the `SELECT` query as you want, allowing you to quickly insert data into a new table from other tables.

<h2 style="color: rgb(241, 90, 36)">Updating data</h2>

The `UPDATE` keyword is used to modify data already existing in the database. When combined with the `WHERE` statement you can accurately target rows or data you want to update. 

The syntax to update rows has the structure:

In [ ]:
UPDATE {table_name}
SET 
    {column_1} = {column_value_1}
    {column_2} = {column_value_2}
    ...
WHERE {condition}

We tell SQL which columns we want to change the values of with the `SET` keyword. Which specifies the column to change and the value to change it to. Notice in the structure of the query we can change multiple columns at once.

The `WHERE` keyword is then used as a conditional to check for the correct rows to update. 

>`WHERE` conditions can match multiple rows, so there's the possibility you might update multiple rows by accident when this wasn't intended. This is more likely to happen when using a `LIKE` statement to pattern match the rows. It's preferable to use the explicit `WHERE column = '<value>'` statements to ensure targeting of the correct information. 

>To avoid this, a careful way to do the update is, first use a `SELECT` and `WHERE` condition to find the rows you will use in the update. If the return rows are the rows you want to update then you can use that condition in the `UPDATE` statement.  

Let's take a look at an an example of applying an `UPDATE`, we'll first `SELECT` the rows to make sure we're targeting the correct rows then update. Let's look at the example of what can go wrong when using a `LIKE` statement. We'll try and update the row with `title = AIRPLANE SIERRA` in the `film` table. Using `LIKE` to find films starting with `AIR`:

In [ ]:
SELECT title 
FROM film
WHERE title LIKE 'AIR%';

This query actually returned two films `AIRPLANE SIERRA` and `AIRPORT POLLOCK`, let's be more specific to avoid updating with incorrect information.

In [ ]:
SELECT title
FROM film
WHERE title = 'AIRPLANE SIERRA';

Perfect, we got an exact match, so we can use this in our `UPDATE` statement. Let's use the condition to update the `rental_rate` and the `release_year` columns. There was an error with the original data, the release year it was actually `2008` and the price has dropped to `2.99`.

In [ ]:
UPDATE film
    SET rental_rate = 2.99,
        release_year = 2010
WHERE 
    title = 'AIRPLANE SIERRA';

Since we confirmed that we were selecting the correct data using the `SELECT/WHERE` query, the `UPDATE` has been applied to the desired row.

`UPDATE` can also be used if the constraints allow, to clear the values in the rows using `NULL`:


In [ ]:
UPDATE film
    SET rental_rate = NULL,
        release_year = NULL
WHERE 
    title = 'AIRPLANE SIERRA';

<h2 style="color: rgb(241, 90, 36)">Removing rows</h2>

The syntax to delete a row from a table is:

In [ ]:
DELETE FROM {table_name} 
WHERE {condition};

Remember you might want to verify you're going to delete the correct rows by first performing a `SELECT` statement before running a `DELETE` statement. In this case we will delete all rows in the `actor` table where the actors `first_name` is `Ben`. Since this query is so specific we can be sure we get the result we want and may not require the `SELECT` first. If you're ever unsure remember to use the `SELECT` first before the `DELETE`. 

In [ ]:
DELETE FROM actor
WHERE first_name = 'Ben';

## Key Takeaways

- Tables can be altered in almost any fashion: dropping rows, updating column, changing data types etc
- `NULL` values in rows can be avoided by setting `DEFAULT` values for rows
- Be careful when changing the data in your table. You can use a `SELECT` statement first to check the correct data is being changed
- `CONSTRAINT`s can be added after a tables creation which can often be the case when you haven't fully modelled your data yet